<a href="https://colab.research.google.com/github/mehdihatami1998/DynamicsOfStructures/blob/main/L07_Response_by_FFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Configure the plotting machinery and the vectorized math libraries.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
import numpy as np

from matplotlib import rcParams 
rcParams['figure.figsize'] = 9,4
black="#404060"

## Response Analysis in the Frequency Domain<br> <small>an Example</small>

##### Samples

In [ ]:
plt.figure(figsize=(9.0,1.1));
plt.subplot(1,2,1); plt.plot((0,1,3),(0,40,0)); plt.xticks((0,1,3)); plt.yticks((0,40))
plt.xlabel('t/s'); plt.ylabel('p(t)/kN'); plt.grid();
plt.subplot(1,2,2); plt.plot((0,1,3,8),(0,40,0,0)); plt.xticks((0,1,3,8)); plt.yticks((0,40))
plt.xlabel('t/s'); plt.ylabel('p(t)/kN'); plt.grid();


We want to replicate the solution obtained for a triangular load using the _Duhamel Integral_.

Our load is 3s long, but we need a stretch of zeros to damp out the response and _simulate_ rest initial conditions. We add zeros to the end of the load up to 
a total duration of 8s. The period of our loading is hence, rather arbitrarily,
$T=8\,{}$s.

In our exercise, we are free to choose the number of _samples per second_,
so we chose 512 sps.

How many samples are there? $N = 8\times512=4096$, note that $N$ is a power of 2.

In [ ]:
T = 8
sps = 512
print("The Nyquist frequency is ", sps/2.0,"Hz")

  
  ##### Load definition

The _array_ `t` contains the times at which our signal was sampled, the load `p` is computed using the library function `where`, syntactically  very similar to `IF` in a spreadsheet

In [ ]:
t = np.arange(0,T,1./sps)
p = np.where(t>3, 0, np.where(t<1, t*40000, 40000*(3-t)/2))

Am I sure that the _list_ p contains the values of the loading?

Let's try to plot p vs t...

In [ ]:
# matplotlib.rcParams['figure.figsize'] = 9,4
plt.plot(t, p, black) ; plt.xlabel("t/s") ; plt.ylabel("p(t)/N")
plt.ylim((-5000,45000));

It looks OK...

##### FFT of the loading

Now, the fast Fourier transform of the sequence p is computed, and given a name, P.

It is customary to denote Fourier pairs by the same letter, the small letter for the time domain representation and the capital letter for the frequency domain representation.

In [ ]:
import numpy.fft as fft

P = fft.fft(p)
iP = fft.ifft(P)

I have computed also the _inverse_ FFT of the FFT of the loading, naming it iP, it is a sequence of complex numbers and here we plot the real and the imaginary part of each component versus time.

In [ ]:
plt.plot(t,np.real(iP),black,t,1*np.imag(iP),'y')
plt.xlabel("t/s") ; plt.ylabel("p(t)/N") ;

It seems OK...

Next, we use a convenience function to compute a sequence of frequencies (in Hertz!) associated with the components of P, the FFT of p. The parameters are the number of points and the sampling interval..

Note that the sequence of frequencies has a discontinuity when the Nyquist frequency
is reached, i.e., the next frequency is the most negative one.

In [ ]:
f = fft.fftfreq(T*sps, 1./sps)
plt.plot(f) ; plt.xlabel('n') ; plt.ylabel('Hz')
plt.xticks(range(0,4097,512)) ; plt.yticks(range(-256,257,64))
plt.grid()

##### Plots of P, the FFT of p

The x axis is streching over the interval $-f_\text{Ny}$, $+f_\text{Ny}$

In [ ]:
plt.plot(f, np.real(P), black, f, np.imag(P), 'b')
plt.xlim(-256,256) ; plt.xticks(range(-256,257,64))
plt.xlabel("f/Hz") ; plt.ylabel("P(f)") ;

The plot above is not much clear, because the frequency components are significantly different from zero only in a narrow range of frequencies around the origin.

In the next 3 plots we zoom near the origin of the frequency axis to have a bit more of detail. There are 3 plots, first the absolute value of P vs f, then the real part and finally the imaginary part of P, versus f.

In [ ]:
plt.plot(f, np.abs(P), black)
plt.xticks(range(-4,5,2))
plt.xlabel("f/Hz") ; plt.ylabel("abs(P(f))")
plt.xlim(-4, 4) ; plt.ylim(-0.2E7, 3.3E7);

Not afwully nice, this last plot...the baseline and the missing line
at the left of the zero are artifacts, due do the particular sequence
with which the positive and negative frequencies are arranged in the DFT output.

To obviate these problems we can use the function `fftshift`, that reorders (shifts)
the elements in an array such that the sequence goes from the most negative
frequency to the most positive.

In [ ]:
plt.plot(fft.fftshift(f), fft.fftshift(np.abs(P)), black)
plt.axhline(0, color=black, linewidth=0.25)
plt.xticks(range(-4, 5, 2))
plt.xlabel("f/Hz") ; plt.ylabel("abs(P(f))")
plt.xlim(-4, 4) ; plt.ylim(-0.2E7, 3.3E7);

and now the other two plots I promised,

In [ ]:
plt.plot(fft.fftshift(f), fft.fftshift(np.real(P)), black)
plt.axhline(0, color=black, linewidth=0.25)
plt.xticks(range(-4, 5, 2))
plt.xlabel("f/Hz") ; plt.ylabel("real(P(f))")
plt.xlim(-4, 4) ; plt.ylim(-3.3E7, 3.3E7);

In [ ]:
plt.plot(fft.fftshift(f), fft.fftshift(np.imag(P)), black)
plt.axhline(0, color=black, linewidth=0.25)
plt.xticks(range(-4, 5, 2))
plt.xlabel("f/Hz") ; plt.ylabel("imag(P(f))")
plt.xlim(-4, 4) ; plt.ylim(-3.3E7, 3.3E7);

##### The response function

Until now, we did without the SDOF, now it's time to describe it and derive its _response function_.

All the parameters are the same as in the excel example, we compute k because we need it to normalize the response.

In [ ]:
z = 0.1; fn = 1/0.6 ; m =6E5 ; wn = fn*2*np.pi ; k = m*wn**2

def H(f, z, fn):
    b = f/fn
    return 1./((1-b*b)+1j*(2*z*b))

As usual, we plot the response function, or rather the absolute value of, against a short span of the frequency axis, centered about the origin, to show the details of the response function itself.

In [ ]:
plt.plot(fft.fftshift(f), fft.fftshift(np.abs(H(f, z, fn))))
plt.xlabel("f/Hz") ; plt.xlim(-8, 8) ; plt.ylabel("H(f)") ;

##### Computing the response

The FFT of the response is computed multiplying, term by term, P by the _transfer_ function, then we compute the IFFT of X to obtain x, the _time domain representation_ of the response.

In [ ]:
X = P * H(f, z, fn)
x = fft.ifft(X)/k

Note that the response function is _periodic_ with period $T=8\,{}$s.

In the end, we remain with the task of plotting the response function, that is the real part of `x`. Just to be certain we plot also the imaginary part of `x`, so we can be sure that it is negligible with respect to the real part

In [ ]:
plt.plot(t, 1000*np.real(x))
plt.xlabel("t/s") ; plt.ylabel(r"$\Re(x)/$mm");
plt.axhline(0, color=black, linewidth=0.25);

In [ ]:
plt.plot(t, 1E18*np.imag(x), linewidth=0.33) # I'm plotting attometres!
plt.axhline(0, color=black, linewidth=0.25)
plt.xlabel("t/s") ; plt.ylabel(r"$\Im(x)/$am");

  ##### The zero trail

The importance of the zero trail to _adjust_ for initial rest condition cannot be
underestimated. The length required depends, of course, on how much damped our system is,
the lesser the damping, the longer the time required to damp out the response.

Lets try to see what happens if we go from $\zeta=0.10$ to $\zeta=0.01$:

In [ ]:
X = P * H(f, 0.01, fn)
x = fft.ifft(X)/k
plt.plot(t, 1000*np.real(x))
plt.axhline(0, color=black, linewidth=0.25)
plt.xlabel("t/s") ; plt.ylabel(r"$\Re(x)/$mm");